In [0]:
from delta.tables import *

from pyspark.sql.types import *

In [0]:
DeltaTable.createOrReplace(spark)\
          .tableName('employee_demo')\
          .addColumn('emp_id',IntegerType(),False)\
          .addColumn('emp_name',StringType(),False)\
          .addColumn('gender',StringType(),False)\
          .addColumn('age',IntegerType(),False)\
          .addColumn('sal',IntegerType(),True)\
          .addColumn('dept',StringType(),True)\
          .property("description","for practise purpose")\
          .location("/FilseStore/tables/delta/practice_delta")\
          .execute()

Out[2]: <delta.tables.DeltaTable at 0x7f6b5c341a60>

In [0]:
%sql 

select * from employee_demo

emp_id,emp_name,gender,age,sal,dept
2,sahithi,f,25,15,sales
1,varun,M,24,13,IT


#SQL STYLE INSERT

In [0]:
%sql
insert into employee_demo values(1,'varun','M',24,13,"IT")

num_affected_rows,num_inserted_rows
1,1


In [0]:
display(spark.sql("select * from employee_demo"))

emp_id,emp_name,gender,age,sal,dept
2,sahithi,f,25,15,sales
1,varun,M,24,13,IT
1,varun,M,24,13,IT


#USING DATAFRAME

In [0]:
employee_data=[(2,'sahithi','f',25,15,"sales")] 

emp_schema=StructType(
                         [
                            StructField('emp_id',IntegerType(),False),
                            StructField('emp_name',StringType(),False),
                            StructField('gender',StringType(),False),
                            StructField('age',IntegerType(),False),
                            StructField('sal',IntegerType(),False),
                            StructField('dept',StringType(),True)
                         ]
                     )

emp_df=spark.createDataFrame(employee_data,emp_schema)

In [0]:
emp_df.write.format('delta').mode('append').saveAsTable('employee_demo')

In [0]:
%sql

select * from employee_demo

emp_id,emp_name,gender,age,sal,dept
2,sahithi,f,25,15,sales
2,sahithi,f,25,15,sales
1,varun,M,24,13,IT
1,varun,M,24,13,IT


#dataFrame InsertInto

In [0]:
employee_data=[(3,'kusuma','f',24,11,"hr")] 

emp_schema=StructType(
                         [
                            StructField('emp_id',IntegerType(),False),
                            StructField('emp_name',StringType(),False),
                            StructField('gender',StringType(),False),
                            StructField('age',IntegerType(),False),
                            StructField('sal',IntegerType(),False),
                            StructField('dept',StringType(),True)
                         ]
                     )

emp_df_1=spark.createDataFrame(employee_data,emp_schema)

In [0]:
emp_df_1.write.insertInto('employee_demo',overwrite=False)

In [0]:
%sql 

select * from employee_demo

emp_id,emp_name,gender,age,sal,dept
2,sahithi,f,25,15,sales
2,sahithi,f,25,15,sales
3,kusuma,f,24,11,hr
1,varun,M,24,13,IT
1,varun,M,24,13,IT


#Inserting using TempView


In [0]:
emp_df_1.createOrReplaceTempView('delta_insert_data')

In [0]:
%sql 

insert into employee_demo 
select * from delta_insert_data;

num_affected_rows,num_inserted_rows
1,1


In [0]:
delta_table_instance_1=DeltaTable.forName(spark,'employee_demo')

display(delta_table_instance_1.toDF())

emp_id,emp_name,gender,age,sal,dept
2,sahithi,f,25,15,sales
2,sahithi,f,25,15,sales
3,kusuma,f,24,11,hr
3,kusuma,f,24,11,hr
1,varun,M,24,13,IT
1,varun,M,24,13,IT


In [0]:
delta_table_instance_1.history().show()

+-------+-------------------+----------------+--------------------+--------------------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|          userId|            userName|           operation| operationParameters| job|          notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+----------------+--------------------+--------------------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|     12|2024-06-05 20:32:16|8432478468657958|paruchuri7781@gma...|               WRITE|{mode -> Append, ...|null|{4188357405012316}|0605-202106-5nxmim3d|         11|WriteSerializable|         true|{numFiles -> 1, n...|        null|Databr